In [1]:
import sys
sys.path.append('/home/infres/slabbi/Master_thesis/lib/python3.8/site-packages')

In [2]:
import torch
X_train = torch.FloatTensor([0., 1., 2.]).cuda()
X_train.is_cuda

True

In [3]:
torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.current_device()
torch.cuda.device(2)
torch.cuda.get_device_name(0)

'Tesla V100-PCIE-16GB'

In [4]:
!nvidia-smi

Thu Aug 24 08:36:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   26C    P0    36W / 250W |    427MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:AF:00.0 Off |                    0 |
| N/A   

In [4]:
from laplace.baselaplace import KronLaplace
from laplace.curvature import AsdlGGN
import torch
from copy import deepcopy
import numpy as np
from torch.nn.utils.convert_parameters import vector_to_parameters
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ExponentialLR, CosineAnnealingLR
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.utils import parameters_to_vector
from torch.distributions import Normal
import matplotlib.pyplot as plt
import math
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as td
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange

from laplace.baselaplace import KronLaplace
from laplace.curvature import AsdlGGN
from PBB.pbb.utils import runexp
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

BATCH_SIZE = 64
TRAIN_EPOCHS = 10
DELTA = 0.025
DELTA_TEST = 0.01
PRIOR = 'learnt'

SIGMAPRIOR = 0.03

PMIN = 1e-5
KL_PENALTY = 0.1
LEARNING_RATE = 0.001
MOMENTUM = 0.95
LEARNING_RATE_PRIOR = 0.005
MOMENTUM_PRIOR = 0.99

# note the number of MC samples used in the paper is 150.000, which usually takes a several hours to compute
MC_SAMPLES = 100

True


# Fquad

In [5]:
net = runexp('mnist', 'fquad', PRIOR, 'cnn', SIGMAPRIOR, PMIN, LEARNING_RATE, MOMENTUM, LEARNING_RATE_PRIOR, MOMENTUM_PRIOR, delta=DELTA, delta_test=DELTA_TEST, mc_samples=MC_SAMPLES, train_epochs=TRAIN_EPOCHS, device=DEVICE, prior_epochs=5, perc_train=1.0, perc_prior=0.5, verbose=True, dropout_prob=0)

 20%|████████████████████████▏                                                                                                | 1/5 [00:05<00:20,  5.02s/it]

-Epoch 0.00000, Train loss: 0.94174, Train err:  0.23747



 40%|████████████████████████████████████████████████▍                                                                        | 2/5 [00:09<00:13,  4.55s/it]

-Epoch 1.00000, Train loss: 0.17680, Train err:  0.05097



 60%|████████████████████████████████████████████████████████████████████████▌                                                | 3/5 [00:13<00:08,  4.44s/it]

...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:22<00:00,  4.41s/it]

-Epoch 4.00000, Train loss: 0.02975, Train err:  0.00920


-Prior: Test loss: 1.93872, Test err:  0.01480


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 21.92it/s]

***Final results***
Objective, Dataset, Sigma, pmin, LR, momentum, LR_prior, momentum_prior, kl_penalty, dropout, Obj_train, Risk_CE, Risk_01, KL, Train NLL loss, Train 01 error, Stch loss, Stch 01 error, Post mean loss, Post mean 01 error, Ens loss, Ens 01 error, 01 error prior net, perc_train, perc_prior
fquad, mnist, 0.03000, 0.00001, 0.00100, 0.95000, 0.00500, 0.99000,  1.00000, 0.00000, 0.07740, 0.07735, 0.10855, 0.00001, 0.07219, 0.10252, 0.00515, 0.01710, 0.00310, 0.01390, 0.00010, 0.01540, 0.01480, 1.00000, 0.50000


In [6]:
from PBB.pbb.models import CNNet4l_from_PBB_net4l
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj

sigma_noise = 1
perc_prior=0.2
backend=AsdlGGN
batch_size = 250
train_epochs = 5
delta = 0.025
delta_test = 0.01
prior = 'learnt'
prior_dist='gaussian'
sigma_prior = 0.03
initial_lamb=6.0
kl_penalty = 0.1
learning_rate = 0.01
momentum = 0.95
rho_prior = math.log(math.exp(sigma_prior)-1.0)
pmin = 1e-5
mc_samples = 15
perc_train=1.0
perc_prior=0.2
layers=9
prior_epochs=10
beta = 10**5
name_data = 'mnist'
train, test = loaddataset(name_data)
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
objective = 'practical_bayes_deep'
learning_rate_prior=0.01
momentum_prior=0.95
verbose=True
verbose_test=True
samples_ensemble=4
sigma_noise = 1
model = 'cnn'


net0 = CNNet4l_from_PBB_net4l(net).to(DEVICE)
train_loader, test_loader, valid_loader, val_bound_one_batch, _, val_bound = loadbatches(
    train, test, loader_kargs, batch_size, prior=True, perc_train=perc_train, perc_prior=perc_prior)
la = KronLaplace(net0, 'classification',  prior_mean = torch.nn.utils.parameters_to_vector(net0.parameters()),sigma_noise=sigma_noise,
                 prior_precision=perc_prior,   backend=backend)
la.fit(train_loader)

samples = torch.randn(2, la.n_params, device=DEVICE)
samples = sigma_prior * la.posterior_precision.bmm(samples, exponent=-1)
theta =  la.mean.reshape(1, la.n_params) + samples.reshape(2, la.n_params)

toolarge = False
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)
posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

bound = PBBobj('practical_bayes_deep', pmin, classes, delta,
        delta_test, mc_samples, kl_penalty, DEVICE, n_posterior = posterior_n_size, n_bound=bound_n_size , theta = theta[0], laplace = la)

train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = computeRiskCertificates(net, toolarge, bound, device=DEVICE,
                                                                                        lambda_var=1, train_loader=val_bound,
                                                                                        whole_train=val_bound_one_batch)
print(objective, train_obj)



torch.Size([32, 1, 3, 3])
torch.Size([32, 1, 3, 3])


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 192/192 [00:09<00:00, 20.70it/s]

trace tensor(3.0668e+08, device='cuda:1')
det tensor(-1634524.1250, device='cuda:1')
empirical risk 0.3980053933727083
tensor(13.9767, device='cuda:1')
practical_bayes_deep 13.976672172546387


In [5]:
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj

train, test = loaddataset('mnist')
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
train_loader, test_loader, _, val_bound_one_batch, _, val_bound = loadbatches(
                train, test, loader_kargs, BATCH_SIZE, prior=False)
posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

toolarge = True
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)
bounds = ['fquad', 'flamb', 'fclassic', 'bbb']
for objective in bounds:
    bound = PBBobj(objective, PMIN, 10, DELTA,
                    DELTA_TEST, MC_SAMPLES, KL_PENALTY, DEVICE, n_posterior = posterior_n_size, n_bound=bound_n_size, evaluate =True)
    train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = computeRiskCertificates(net, toolarge, bound, device=DEVICE,
                                                                                            lambda_var=1, train_loader=val_bound,
                                                                                            whole_train=val_bound_one_batch)
    print(objective, train_obj)



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [14:57<00:00,  1.04it/s]


fquad 0.07262290269136429


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [15:02<00:00,  1.04it/s]


flamb 0.13579825754517522


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [14:34<00:00,  1.07it/s]


fclassic 0.07685442268848419


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [14:51<00:00,  1.05it/s]

bbb 0.06772398948669434


In [7]:
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj
from PBB.pbb.models import NNet4l, CNNet4l, ProbNNet4l, ProbCNNet4l, ProbCNNet9l, CNNet9l, CNNet13l, ProbCNNet13l, ProbCNNet15l, CNNet15l, trainNNet, testNNet, Lambda_var, trainPNNet, computeRiskCertificates, testPosteriorMean, testStochastic, testEnsemble, custom_weights, CNNet4l_no_activation, trainNNet_cross_entropy, testNNet_cross_entropy, nll_loss_NNet_train_set , ProbCNNet4l_no_activation, ProbNNet2l_no_activation, NNet2l_no_activation
from laplace.baselaplace import KronLaplace
from laplace.curvature import AsdlGGN

backend=AsdlGGN
train, test = loaddataset('mnist')
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
train_loader, test_loader, _, val_bound_one_batch, _, val_bound = loadbatches(
                train, test, loader_kargs, BATCH_SIZE, prior=False)
batch_size = 250
perc_train=1.0
perc_prior=0.2
learning_rate_prior=0.01
momentum_prior=0.95
prior_epochs=10
verbose=True
verbose_test=True
net0 = CNNet4l_no_activation().to(DEVICE)
train_loader, test_loader, valid_loader, val_bound_one_batch, _, val_bound = loadbatches(
    train, test, loader_kargs, batch_size, prior=True, perc_train=perc_train, perc_prior=perc_prior)
optimizer = optim.SGD(net0.parameters(), lr=learning_rate_prior, momentum=momentum_prior)
for epoch in trange(prior_epochs):
    trainNNet_cross_entropy(net0, optimizer, epoch, valid_loader,
              device=DEVICE, verbose=verbose)
errornet0 = testNNet_cross_entropy(net0, test_loader, device=DEVICE)

posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

toolarge = True
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)

la = KronLaplace(net0, 'classification', backend=backend)
la.fit(train_loader)

bound = PBBobj('loss_curvature_bayes', PMIN, 10, DELTA,
                DELTA_TEST, MC_SAMPLES, KL_PENALTY, DEVICE, n_posterior = posterior_n_size, n_bound=bound_n_size,net0 = net0, H = la.H, laplace = la, evaluate =True)
train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = computeRiskCertificates(net, toolarge, bound, device=DEVICE,
                                                                                        lambda_var=1, train_loader=val_bound,
                                                                                        whole_train=val_bound_one_batch, compute_expectation = True)
print('loss_curvature_bayes', train_obj)

 10%|████████████                                                                                                            | 1/10 [00:01<00:16,  1.87s/it]

-Epoch 0.00000, Train loss: 1.18030, Train err:  0.31183



 20%|████████████████████████                                                                                                | 2/10 [00:03<00:15,  1.88s/it]

-Epoch 1.00000, Train loss: 0.33554, Train err:  0.10075



 30%|████████████████████████████████████                                                                                    | 3/10 [00:05<00:13,  1.87s/it]

...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:18<00:00,  1.81s/it]

-Epoch 9.00000, Train loss: 0.01528, Train err:  0.00408


-Prior: Test loss: 2.65070, Test err:  0.01920


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 192/192 [02:49<00:00,  1.13it/s]

loss_curvature_bayes 647200.5625


In [12]:
marglik = -la.log_marginal_likelihood(perc_prior) / train_size
print(marglik)

tensor(2.3161, device='cuda:1')


# Flambda

In [9]:
net = runexp('mnist', 'flamb', PRIOR, 'cnn', SIGMAPRIOR, PMIN, LEARNING_RATE, MOMENTUM, LEARNING_RATE_PRIOR, MOMENTUM_PRIOR, delta=DELTA, delta_test=DELTA_TEST, mc_samples=MC_SAMPLES, train_epochs=20, device=DEVICE, prior_epochs=10, perc_train=1.0, perc_prior=0.5, verbose=True, dropout_prob=0)

 10%|████████████                                                                                                            | 1/10 [00:04<00:38,  4.26s/it]

-Epoch 0.00000, Train loss: 0.94174, Train err:  0.23747



 20%|████████████████████████                                                                                                | 2/10 [00:08<00:33,  4.21s/it]

-Epoch 1.00000, Train loss: 0.17681, Train err:  0.05103



 30%|████████████████████████████████████                                                                                    | 3/10 [00:12<00:29,  4.19s/it]

...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:41<00:00,  4.19s/it]

-Epoch 9.00000, Train loss: 0.00232, Train err:  0.00043


-Prior: Test loss: 2.30565, Test err:  0.01220


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 25.74it/s]

***Final results***
Objective, Dataset, Sigma, pmin, LR, momentum, LR_prior, momentum_prior, kl_penalty, dropout, Obj_train, Risk_CE, Risk_01, KL, Train NLL loss, Train 01 error, Stch loss, Stch 01 error, Post mean loss, Post mean 01 error, Ens loss, Ens 01 error, 01 error prior net, perc_train, perc_prior
flamb, mnist, 0.03000, 0.00001, 0.00100, 0.95000, 0.00500, 0.99000,  1.00000, 0.00000, 0.14935, 0.08018, 0.10338, 0.00003, 0.07466, 0.09718, 0.00586, 0.01470, 0.00214, 0.01110, 0.00019, 0.01350, 0.01220, 1.00000, 0.50000


In [14]:
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj

train, test = loaddataset('mnist')
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
train_loader, test_loader, _, val_bound_one_batch, _, val_bound = loadbatches(
                train, test, loader_kargs, BATCH_SIZE, prior=False)
posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

toolarge = True
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)
bounds = ['fquad', 'flamb', 'fclassic', 'bbb']
for objective in bounds:
    bound = PBBobj(objective, PMIN, 10, DELTA,
                    DELTA_TEST, MC_SAMPLES, KL_PENALTY, DEVICE, n_posterior = posterior_n_size, n_bound=bound_n_size, evaluate =True)
    train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = computeRiskCertificates(net, toolarge, bound, device=DEVICE,
                                                                                            lambda_var=1, train_loader=val_bound,
                                                                                            whole_train=val_bound_one_batch)
    print(objective, train_obj)



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [14:34<00:00,  1.07it/s]


fquad 0.07217336446046829


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [13:56<00:00,  1.12it/s]


flamb 0.13487084393312343


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [13:27<00:00,  1.16it/s]


fclassic 0.07645564526319504


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [13:29<00:00,  1.16it/s]

bbb 0.06726741790771484


In [10]:
from PBB.pbb.models import CNNet4l_from_PBB_net4l
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj

sigma_noise = 1
perc_prior=0.2
backend=AsdlGGN
batch_size = 250
train_epochs = 5
delta = 0.025
delta_test = 0.01
prior = 'learnt'
prior_dist='gaussian'
sigma_prior = 0.03
initial_lamb=6.0
kl_penalty = 0.1
learning_rate = 0.01
momentum = 0.95
rho_prior = math.log(math.exp(sigma_prior)-1.0)
pmin = 1e-5
mc_samples = 15
perc_train=1.0
perc_prior=0.2
layers=9
prior_epochs=10
beta = 10**5
name_data = 'mnist'
train, test = loaddataset(name_data)
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
objective = 'practical_bayes_deep'
learning_rate_prior=0.01
momentum_prior=0.95
verbose=True
verbose_test=True
samples_ensemble=4
sigma_noise = 1
model = 'cnn'


net0 = CNNet4l_from_PBB_net4l(net).to(DEVICE)
train_loader, test_loader, valid_loader, val_bound_one_batch, _, val_bound = loadbatches(
    train, test, loader_kargs, batch_size, prior=True, perc_train=perc_train, perc_prior=perc_prior)
la = KronLaplace(net0, 'classification',  prior_mean = torch.nn.utils.parameters_to_vector(net0.parameters()),sigma_noise=sigma_noise,
                 prior_precision=perc_prior,   backend=backend)
la.fit(train_loader)

samples = torch.randn(2, la.n_params, device=DEVICE)
samples = sigma_prior * la.posterior_precision.bmm(samples, exponent=-1)
theta =  la.mean.reshape(1, la.n_params) + samples.reshape(2, la.n_params)

toolarge = False
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)
posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

bound = PBBobj('practical_bayes_deep', pmin, classes, delta,
        delta_test, mc_samples, kl_penalty, DEVICE, n_posterior = posterior_n_size, n_bound=bound_n_size , theta = theta[0], laplace = la)

train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = computeRiskCertificates(net, toolarge, bound, device=DEVICE,
                                                                                        lambda_var=1, train_loader=val_bound,
                                                                                        whole_train=val_bound_one_batch)
print(objective, train_obj)



torch.Size([32, 1, 3, 3])
torch.Size([32, 1, 3, 3])


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 192/192 [00:09<00:00, 20.53it/s]

trace tensor(3.0668e+08, device='cuda:1')
det tensor(-1634524.1250, device='cuda:1')
empirical risk 0.39373327413656944
tensor(13.9724, device='cuda:1')
practical_bayes_deep 13.972399711608887


In [15]:
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj
from PBB.pbb.models import NNet4l, CNNet4l, ProbNNet4l, ProbCNNet4l, ProbCNNet9l, CNNet9l, CNNet13l, ProbCNNet13l, ProbCNNet15l, CNNet15l, trainNNet, testNNet, Lambda_var, trainPNNet, computeRiskCertificates, testPosteriorMean, testStochastic, testEnsemble, custom_weights, CNNet4l_no_activation, trainNNet_cross_entropy, testNNet_cross_entropy, nll_loss_NNet_train_set , ProbCNNet4l_no_activation, ProbNNet2l_no_activation, NNet2l_no_activation
from laplace.baselaplace import KronLaplace
from laplace.curvature import AsdlGGN

backend=AsdlGGN
train, test = loaddataset('mnist')
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
train_loader, test_loader, _, val_bound_one_batch, _, val_bound = loadbatches(
                train, test, loader_kargs, BATCH_SIZE, prior=False)
batch_size = 250
perc_train=1.0
perc_prior=0.2
learning_rate_prior=0.01
momentum_prior=0.95
prior_epochs=10
verbose=True
verbose_test=True
net0 = CNNet4l_no_activation().to(DEVICE)
train_loader, test_loader, valid_loader, val_bound_one_batch, _, val_bound = loadbatches(
    train, test, loader_kargs, batch_size, prior=True, perc_train=perc_train, perc_prior=perc_prior)
optimizer = optim.SGD(net0.parameters(), lr=learning_rate_prior, momentum=momentum_prior)
for epoch in trange(prior_epochs):
    trainNNet_cross_entropy(net0, optimizer, epoch, valid_loader,
              device=DEVICE, verbose=verbose)
errornet0 = testNNet_cross_entropy(net0, test_loader, device=DEVICE)

posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

toolarge = True
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)

la = KronLaplace(net0, 'classification', backend=backend)
la.fit(train_loader)

bound = PBBobj('loss_curvature_bayes', PMIN, 10, DELTA,
                DELTA_TEST, MC_SAMPLES, KL_PENALTY, DEVICE, n_posterior = posterior_n_size, n_bound=bound_n_size,net0 = net0, H = la.H, laplace = la, evaluate =True)
train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = computeRiskCertificates(net, toolarge, bound, device=DEVICE,
                                                                                        lambda_var=1, train_loader=val_bound,
                                                                                        whole_train=val_bound_one_batch, compute_expectation = True)
print('loss_curvature_bayes', train_obj)

 10%|████████████                                                                                                            | 1/10 [00:01<00:15,  1.76s/it]

-Epoch 0.00000, Train loss: 1.18030, Train err:  0.31183



 20%|████████████████████████                                                                                                | 2/10 [00:03<00:13,  1.75s/it]

-Epoch 1.00000, Train loss: 0.33554, Train err:  0.10075



 30%|████████████████████████████████████                                                                                    | 3/10 [00:05<00:12,  1.75s/it]

...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.75s/it]

-Epoch 9.00000, Train loss: 0.01528, Train err:  0.00408


-Prior: Test loss: 2.65070, Test err:  0.01920


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 192/192 [02:48<00:00,  1.14it/s]

loss_curvature_bayes 1847388.125


In [16]:
marglik = -la.log_marginal_likelihood(perc_prior) / train_size
print(marglik)

tensor(2.3161, device='cuda:1')


# Fclassic

In [11]:
net_classique = runexp('mnist', 'fclassic', PRIOR, 'cnn', SIGMAPRIOR, PMIN, LEARNING_RATE, MOMENTUM, LEARNING_RATE_PRIOR, MOMENTUM_PRIOR, delta=DELTA, delta_test=DELTA_TEST, mc_samples=MC_SAMPLES, train_epochs=TRAIN_EPOCHS, device=DEVICE, prior_epochs=5, perc_train=1.0, perc_prior=0.5, verbose=True, dropout_prob=0)

 20%|████████████████████████▏                                                                                                | 1/5 [00:04<00:16,  4.18s/it]

-Epoch 0.00000, Train loss: 0.94177, Train err:  0.23747



 40%|████████████████████████████████████████████████▍                                                                        | 2/5 [00:08<00:12,  4.21s/it]

-Epoch 1.00000, Train loss: 0.17658, Train err:  0.05083



 60%|████████████████████████████████████████████████████████████████████████▌                                                | 3/5 [00:12<00:08,  4.18s/it]

...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:21<00:00,  4.20s/it]

-Epoch 4.00000, Train loss: 0.02924, Train err:  0.00937


-Prior: Test loss: 1.94158, Test err:  0.01440


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 25.66it/s]

***Final results***
Objective, Dataset, Sigma, pmin, LR, momentum, LR_prior, momentum_prior, kl_penalty, dropout, Obj_train, Risk_CE, Risk_01, KL, Train NLL loss, Train 01 error, Stch loss, Stch 01 error, Post mean loss, Post mean 01 error, Ens loss, Ens 01 error, 01 error prior net, perc_train, perc_prior
fclassic, mnist, 0.03000, 0.00001, 0.00100, 0.95000, 0.00500, 0.99000,  1.00000, 0.00000, 0.08179, 0.07770, 0.10905, 0.00000, 0.07264, 0.10314, 0.00527, 0.01740, 0.00333, 0.01390, 0.00011, 0.01500, 0.01440, 1.00000, 0.50000


In [13]:
from PBB.pbb.models import CNNet4l_from_PBB_net4l
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj

sigma_noise = 1
perc_prior=0.2
backend=AsdlGGN
batch_size = 250
train_epochs = 5
delta = 0.025
delta_test = 0.01
prior = 'learnt'
prior_dist='gaussian'
sigma_prior = 0.03
initial_lamb=6.0
kl_penalty = 0.1
learning_rate = 0.01
momentum = 0.95
rho_prior = math.log(math.exp(sigma_prior)-1.0)
pmin = 1e-5
mc_samples = 15
perc_train=1.0
perc_prior=0.2
layers=9
prior_epochs=10
beta = 10**5
name_data = 'mnist'
train, test = loaddataset(name_data)
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
objective = 'practical_bayes_deep'
learning_rate_prior=0.01
momentum_prior=0.95
verbose=True
verbose_test=True
samples_ensemble=4
sigma_noise = 1
model = 'cnn'


net0 = CNNet4l_from_PBB_net4l(net_classique).to(DEVICE)
train_loader, test_loader, valid_loader, val_bound_one_batch, _, val_bound = loadbatches(
    train, test, loader_kargs, batch_size, prior=True, perc_train=perc_train, perc_prior=perc_prior)
la = KronLaplace(net0, 'classification',  prior_mean = torch.nn.utils.parameters_to_vector(net0.parameters()),sigma_noise=sigma_noise,
                 prior_precision=perc_prior,   backend=backend)
la.fit(train_loader)

samples = torch.randn(2, la.n_params, device=DEVICE)
samples = sigma_prior * la.posterior_precision.bmm(samples, exponent=-1)
theta =  la.mean.reshape(1, la.n_params) + samples.reshape(2, la.n_params)

toolarge = False
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)
posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

bound = PBBobj('practical_bayes_deep', pmin, classes, delta,
        delta_test, mc_samples, kl_penalty, DEVICE, n_posterior = posterior_n_size, n_bound=bound_n_size , theta = theta[0], laplace = la)

train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = computeRiskCertificates(net_classique, toolarge, bound, device=DEVICE,
                                                                                        lambda_var=1, train_loader=val_bound,
                                                                                        whole_train=val_bound_one_batch)
print(objective, train_obj)



torch.Size([32, 1, 3, 3])
torch.Size([32, 1, 3, 3])


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 192/192 [00:09<00:00, 20.37it/s]


trace tensor(3.0668e+08, device='cuda:1')
det tensor(-1634524.1250, device='cuda:1')
empirical risk 0.39944946655328767
tensor(13.9781, device='cuda:1')
practical_bayes_deep 13.978116035461426


In [18]:
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj

train, test = loaddataset('mnist')
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
train_loader, test_loader, _, val_bound_one_batch, _, val_bound = loadbatches(
                train, test, loader_kargs, BATCH_SIZE, prior=False)
posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

toolarge = True
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)
bounds = ['fquad', 'flamb', 'fclassic', 'bbb']
for objective in bounds:
    bound = PBBobj(objective, PMIN, 10, DELTA,
                    DELTA_TEST, MC_SAMPLES, KL_PENALTY, DEVICE, n_posterior = posterior_n_size, n_bound=bound_n_size, evaluate =True)
    train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = computeRiskCertificates(net_classique, toolarge, bound, device=DEVICE,
                                                                                            lambda_var=1, train_loader=val_bound,
                                                                                            whole_train=val_bound_one_batch)
    print(objective, train_obj)



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [13:33<00:00,  1.15it/s]


fquad 0.07285448163747787


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [13:34<00:00,  1.15it/s]


flamb 0.13628489809369743


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [13:31<00:00,  1.16it/s]


fclassic 0.07708245515823364


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [13:32<00:00,  1.15it/s]

bbb 0.06796605885028839


In [6]:
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj
from PBB.pbb.models import NNet4l, CNNet4l, ProbNNet4l, ProbCNNet4l, ProbCNNet9l, CNNet9l, CNNet13l, ProbCNNet13l, ProbCNNet15l, CNNet15l, trainNNet, testNNet, Lambda_var, trainPNNet, computeRiskCertificates, testPosteriorMean, testStochastic, testEnsemble, custom_weights, CNNet4l_no_activation, trainNNet_cross_entropy, testNNet_cross_entropy, nll_loss_NNet_train_set , ProbCNNet4l_no_activation, ProbNNet2l_no_activation, NNet2l_no_activation, CNNet4l_from_PBB_net4l
from laplace.baselaplace import KronLaplace
from laplace.curvature import AsdlGGN

backend=AsdlGGN
train, test = loaddataset('mnist')
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
train_loader, test_loader, _, val_bound_one_batch, _, val_bound = loadbatches(
                train, test, loader_kargs, BATCH_SIZE, prior=False)
batch_size = 250
perc_train=1.0
perc_prior=0.2
learning_rate_prior=0.01
momentum_prior=0.95
prior_epochs=10
verbose=True
verbose_test=True
net0 = CNNet4l_no_activation().to(DEVICE)
train_loader, test_loader, valid_loader, val_bound_one_batch, _, val_bound = loadbatches(
    train, test, loader_kargs, batch_size, prior=True, perc_train=perc_train, perc_prior=perc_prior)
optimizer = optim.SGD(net0.parameters(), lr=learning_rate_prior, momentum=momentum_prior)
for epoch in trange(prior_epochs):
    trainNNet_cross_entropy(net0, optimizer, epoch, valid_loader,
              device=DEVICE, verbose=verbose)
errornet0 = testNNet_cross_entropy(net0, test_loader, device=DEVICE)

posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

toolarge = True
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)

la = KronLaplace(net0, 'classification', backend=backend)
la.fit(train_loader)

bound = PBBobj('loss_curvature_bayes', PMIN, 10, DELTA,
                DELTA_TEST, MC_SAMPLES, KL_PENALTY, DEVICE, n_posterior = posterior_n_size, n_bound=bound_n_size,net0 = net0, H = la.H, laplace = la, evaluate =True)
train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = computeRiskCertificates(net_classique, toolarge, bound, device=DEVICE,
                                                                                        lambda_var=1, train_loader=val_bound,
                                                                                        whole_train=val_bound_one_batch, compute_expectation = True)
print('loss_curvature_bayes', train_obj)

 10%|████████████                                                                                                            | 1/10 [00:01<00:17,  1.93s/it]

-Epoch 0.00000, Train loss: 1.18030, Train err:  0.31183



 20%|████████████████████████                                                                                                | 2/10 [00:03<00:14,  1.86s/it]

-Epoch 1.00000, Train loss: 0.33554, Train err:  0.10075



 30%|████████████████████████████████████                                                                                    | 3/10 [00:05<00:12,  1.83s/it]

...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:18<00:00,  1.83s/it]

-Epoch 9.00000, Train loss: 0.01528, Train err:  0.00408


-Prior: Test loss: 2.65070, Test err:  0.01920


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 192/192 [02:48<00:00,  1.14it/s]

loss_curvature_bayes 150789.328125


In [7]:
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj
from PBB.pbb.models import NNet4l, CNNet4l, ProbNNet4l, ProbCNNet4l, ProbCNNet9l, CNNet9l, CNNet13l, ProbCNNet13l, ProbCNNet15l, CNNet15l, trainNNet, testNNet, Lambda_var, trainPNNet, computeRiskCertificates, testPosteriorMean, testStochastic, testEnsemble, custom_weights, CNNet4l_no_activation, trainNNet_cross_entropy, testNNet_cross_entropy, nll_loss_NNet_train_set , ProbCNNet4l_no_activation, ProbNNet2l_no_activation, NNet2l_no_activation, CNNet4l_from_PBB_net4l
from laplace.baselaplace import KronLaplace
from laplace.curvature import AsdlGGN

backend=AsdlGGN
train, test = loaddataset('mnist')
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
train_loader, test_loader, _, val_bound_one_batch, _, val_bound = loadbatches(
                train, test, loader_kargs, BATCH_SIZE, prior=False)
batch_size = 250
perc_train=1.0
perc_prior=0.2
learning_rate_prior=0.01
momentum_prior=0.95
prior_epochs=10
verbose=True
verbose_test=True

posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

toolarge = True
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)

class CNNet4l_from_PBB_net4l(nn.Module):
    """Implementation of a standard Convolutional Neural Network with 4 layers with no activation function at the end
    (used for the experiments on MNIST so it assumes a specific input size and
    number of classes)

    Parameters
    ----------
    dropout_prob : float
        probability of an element to be zeroed.

    device : string
        Device the code will run in (e.g. 'cuda')

    """

    def __init__(self,pnet):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv1.weight = pnet.conv1.weight.mu
        self.conv1.bias = pnet.conv1.bias.mu
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv2.weight = pnet.conv2.weight.mu
        self.conv2.bias = pnet.conv2.bias.mu
        self.fc1 = nn.Linear(9216, 128)
        self.fc1.weight= pnet.fc1.weight.mu 
        self.fc1.bias= pnet.fc1.bias.mu 
        self.fc2 = nn.Linear(128, 10)
        self.fc2.weight= pnet.fc2.weight.mu 
        self.fc2.bias= pnet.fc2.bias.mu 
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

net1 = CNNet4l_from_PBB_net4l(net_classique).to(DEVICE)

# Fclassic

In [14]:
net_classique = runexp('mnist', 'bbb', PRIOR, 'cnn', SIGMAPRIOR, PMIN, LEARNING_RATE, MOMENTUM, LEARNING_RATE_PRIOR, MOMENTUM_PRIOR, delta=DELTA, delta_test=DELTA_TEST, mc_samples=MC_SAMPLES, train_epochs=TRAIN_EPOCHS, device=DEVICE, prior_epochs=5, perc_train=1.0, perc_prior=0.5, verbose=True, dropout_prob=0)

 20%|████████████████████████▏                                                                                                | 1/5 [00:04<00:17,  4.35s/it]

-Epoch 0.00000, Train loss: 0.94177, Train err:  0.23747



 40%|████████████████████████████████████████████████▍                                                                        | 2/5 [00:08<00:12,  4.30s/it]

-Epoch 1.00000, Train loss: 0.17667, Train err:  0.05097



 60%|████████████████████████████████████████████████████████████████████████▌                                                | 3/5 [00:12<00:08,  4.27s/it]

...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:21<00:00,  4.29s/it]

-Epoch 4.00000, Train loss: 0.02940, Train err:  0.00937


-Prior: Test loss: 1.95824, Test err:  0.01460


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 25.68it/s]

***Final results***
Objective, Dataset, Sigma, pmin, LR, momentum, LR_prior, momentum_prior, kl_penalty, dropout, Obj_train, Risk_CE, Risk_01, KL, Train NLL loss, Train 01 error, Stch loss, Stch 01 error, Post mean loss, Post mean 01 error, Ens loss, Ens 01 error, 01 error prior net, perc_train, perc_prior
bbb, mnist, 0.03000, 0.00001, 0.00100, 0.95000, 0.00500, 0.99000,  1.00000, 0.00000, 0.07096, 0.07912, 0.10941, 0.00033, 0.07063, 0.09956, 0.00463, 0.01560, 0.00299, 0.01280, 0.00008, 0.01400, 0.01460, 1.00000, 0.50000


In [15]:
from PBB.pbb.models import CNNet4l_from_PBB_net4l
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj

sigma_noise = 1
perc_prior=0.2
backend=AsdlGGN
batch_size = 250
train_epochs = 5
delta = 0.025
delta_test = 0.01
prior = 'learnt'
prior_dist='gaussian'
sigma_prior = 0.03
initial_lamb=6.0
kl_penalty = 0.1
learning_rate = 0.01
momentum = 0.95
rho_prior = math.log(math.exp(sigma_prior)-1.0)
pmin = 1e-5
mc_samples = 15
perc_train=1.0
perc_prior=0.2
layers=9
prior_epochs=10
beta = 10**5
name_data = 'mnist'
train, test = loaddataset(name_data)
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
objective = 'practical_bayes_deep'
learning_rate_prior=0.01
momentum_prior=0.95
verbose=True
verbose_test=True
samples_ensemble=4
sigma_noise = 1
model = 'cnn'


net0 = CNNet4l_from_PBB_net4l(net_classique).to(DEVICE)
train_loader, test_loader, valid_loader, val_bound_one_batch, _, val_bound = loadbatches(
    train, test, loader_kargs, batch_size, prior=True, perc_train=perc_train, perc_prior=perc_prior)
la = KronLaplace(net0, 'classification',  prior_mean = torch.nn.utils.parameters_to_vector(net0.parameters()),sigma_noise=sigma_noise,
                 prior_precision=perc_prior,   backend=backend)
la.fit(train_loader)

samples = torch.randn(2, la.n_params, device=DEVICE)
samples = sigma_prior * la.posterior_precision.bmm(samples, exponent=-1)
theta =  la.mean.reshape(1, la.n_params) + samples.reshape(2, la.n_params)

toolarge = False
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)
posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

bound = PBBobj('practical_bayes_deep', pmin, classes, delta,
        delta_test, mc_samples, kl_penalty, DEVICE, n_posterior = posterior_n_size, n_bound=bound_n_size , theta = theta[0], laplace = la)

train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = computeRiskCertificates(net_classique, toolarge, bound, device=DEVICE,
                                                                                        lambda_var=1, train_loader=val_bound,
                                                                                        whole_train=val_bound_one_batch)
print(objective, train_obj)


torch.Size([32, 1, 3, 3])
torch.Size([32, 1, 3, 3])


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 192/192 [00:09<00:00, 20.51it/s]

trace tensor(3.0668e+08, device='cuda:1')
det tensor(-1634524.1250, device='cuda:1')
empirical risk 0.3904977012811917
tensor(13.9692, device='cuda:1')
practical_bayes_deep 13.969164848327637


In [9]:
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj

train, test = loaddataset('mnist')
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
train_loader, test_loader, _, val_bound_one_batch, _, val_bound = loadbatches(
                train, test, loader_kargs, BATCH_SIZE, prior=False)
posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

toolarge = True
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)
bounds = ['fquad', 'flamb', 'fclassic', 'bbb']
for objective in bounds:
    bound = PBBobj(objective, PMIN, 10, DELTA,
                    DELTA_TEST, MC_SAMPLES, KL_PENALTY, DEVICE, n_posterior = posterior_n_size, n_bound=bound_n_size, evaluate =True)
    train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = computeRiskCertificates(net_classique, toolarge, bound, device=DEVICE,
                                                                                            lambda_var=1, train_loader=val_bound,
                                                                                            whole_train=val_bound_one_batch)
    print(objective, train_obj)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [14:30<00:00,  1.08it/s]


fquad 0.07162467390298843


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [14:25<00:00,  1.08it/s]


flamb 0.1329590235711316


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [14:26<00:00,  1.08it/s]


fclassic 0.07630749046802521


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [14:24<00:00,  1.08it/s]

bbb 0.06632191687822342


In [13]:
from PBB.pbb.data import loaddataset, loadbatches
from PBB.pbb.models  import computeRiskCertificates
from PBB.pbb.bounds import PBBobj
from PBB.pbb.models import NNet4l, CNNet4l, ProbNNet4l, ProbCNNet4l, ProbCNNet9l, CNNet9l, CNNet13l, ProbCNNet13l, ProbCNNet15l, CNNet15l, trainNNet, testNNet, Lambda_var, trainPNNet, computeRiskCertificates, testPosteriorMean, testStochastic, testEnsemble, custom_weights, CNNet4l_no_activation, trainNNet_cross_entropy, testNNet_cross_entropy, ProbCNNet4l_no_activation, ProbNNet2l_no_activation, NNet2l_no_activation, CNNet4l_from_PBB_net4l
from laplace.baselaplace import KronLaplace
from laplace.curvature import AsdlGGN

backend=AsdlGGN
train, test = loaddataset('mnist')
loader_kargs = {'num_workers': 1,
                'pin_memory': True} if torch.cuda.is_available() else {}
train_loader, test_loader, _, val_bound_one_batch, _, val_bound = loadbatches(
                train, test, loader_kargs, BATCH_SIZE, prior=False)
batch_size = 250
perc_train=1.0
perc_prior=0.2
learning_rate_prior=0.01
momentum_prior=0.95
prior_epochs=10
verbose=True
verbose_test=True
net0 = CNNet4l_no_activation().to(DEVICE)
train_loader, test_loader, valid_loader, val_bound_one_batch, _, val_bound = loadbatches(
    train, test, loader_kargs, batch_size, prior=True, perc_train=perc_train, perc_prior=perc_prior)
optimizer = optim.SGD(net0.parameters(), lr=learning_rate_prior, momentum=momentum_prior)
for epoch in trange(prior_epochs):
    trainNNet_cross_entropy(net0, optimizer, epoch, valid_loader,
              device=DEVICE, verbose=verbose)
errornet0 = testNNet_cross_entropy(net0, test_loader, device=DEVICE)

posterior_n_size = len(train_loader.dataset)
bound_n_size = len(val_bound.dataset)

toolarge = True
train_size = len(train_loader.dataset)
classes = len(train_loader.dataset.classes)

la = KronLaplace(net0, 'classification', backend=backend)
la.fit(train_loader)

bound = PBBobj('loss_curvature_bayes', PMIN, 10, DELTA,
                DELTA_TEST, MC_SAMPLES, KL_PENALTY, DEVICE, n_posterior = posterior_n_size, n_bound=bound_n_size,net0 = net0, H = la.H, laplace = la, evaluate =True)
train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = computeRiskCertificates(net_classique, toolarge, bound, device=DEVICE,
                                                                                        lambda_var=1, train_loader=val_bound,
                                                                                        whole_train=val_bound_one_batch, compute_expectation = True)
print('loss_curvature_bayes', train_obj)

 10%|████████████                                                                                                            | 1/10 [00:01<00:15,  1.74s/it]

-Epoch 0.00000, Train loss: 1.18024, Train err:  0.31150



 20%|████████████████████████                                                                                                | 2/10 [00:03<00:13,  1.73s/it]

-Epoch 1.00000, Train loss: 0.33485, Train err:  0.10033



 30%|████████████████████████████████████                                                                                    | 3/10 [00:05<00:12,  1.73s/it]

...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.74s/it]

-Epoch 9.00000, Train loss: 0.01515, Train err:  0.00400


-Prior: Test loss: 2.60538, Test err:  0.01900


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 192/192 [02:57<00:00,  1.08it/s]

loss_curvature_bayes 20817796.0
